## Homework - Exercise 3
Your goal is to join two collections by key. One collection (A) is "big data" size and has to be processed on many machines and the other one (B) is small enough to fit in memory of a single machine. Implement the join by using RDD with the first collection and a broadcast variable with the second one.

In [1]:
#@title PySpark installation and imports

!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet
!apt install openjdk-8-jdk-headless &> /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
#@title Sample Data
A = [(1,2), (1,3), (2,2)]  # A(X,Y)
B = [(2,4), (3,4), (3,5)]  # B(Y,Z)
AjoinB = [(1,2,4), (1,3,4), (1,3,5), (2,2,4)]  # A(X,Y) join B(Y,Z) on Y

print(f"A={A}")
print(f"B={B}")
print(f"AjoinB={AjoinB}")

A=[(1, 2), (1, 3), (2, 2)]
B=[(2, 4), (3, 4), (3, 5)]
AjoinB=[(1, 2, 4), (1, 3, 4), (1, 3, 5), (2, 2, 4)]


In [4]:
#@title Solution
conf = SparkConf().set('spark.ui.port', '4050').setAppName("mlibs").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

def broadcast_join(large_rdd, small_list):
    small_dict = {}
    for key, value in small_list:
        if key not in small_dict:
            small_dict[key] = []
        small_dict[key].append(value)

    broadcast_small_dict = sc.broadcast(small_dict)
    joined_rdd = large_rdd.flatMap(lambda x: [(x[0], x[1], val) for val in broadcast_small_dict.value.get(x[1], [])])

    return joined_rdd

A_rdd = sc.parallelize(A) #Parallelize 'big' dataset
result_rdd = broadcast_join(A_rdd, B)

final_result = result_rdd.collect()
print('Result:')
print(list(final_result))
print('Correct result:')
print(AjoinB)

sc.stop()

Result:
[(1, 2, 4), (1, 3, 4), (1, 3, 5), (2, 2, 4)]
Correct result:
[(1, 2, 4), (1, 3, 4), (1, 3, 5), (2, 2, 4)]
